# F1Ops Visualization & Reporting

This notebook demonstrates visualization techniques for F1 logistics data using Folium and Plotly.

**Version**: 0.1 (Feb 2020)
**Libraries**: Folium 0.10, Plotly 4.5

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import plotly.graph_objects as go
import plotly.express as px

from f1ops.data_loader import get_european_races, load_circuits
from f1ops.geo import build_season_legs
from f1ops.cost import calculate_leg_cost, calculate_season_cost
from f1ops.emissions import calculate_leg_emissions
from f1ops.viz import create_route_map, create_distance_chart
from f1ops.config import DEFAULT_COST_PARAMS, DEFAULT_EMISSIONS_PARAMS

%matplotlib inline
sns.set_style('whitegrid')

## 1. Load Data

In [ ]:
# Load 2019 season
races_2019 = get_european_races(2019)
legs = build_season_legs(races_2019)
circuits = load_circuits()

print(f"Loaded: {len(races_2019)} races, {len(legs)} legs, {len(circuits)} circuits")

## 2. Interactive Map with Folium

Folium provides interactive maps using Leaflet.js, perfect for geographic visualizations.

In [ ]:
# Create base map centered on Europe
m = folium.Map(
    location=[48.0, 10.0],  # Center on Europe
    zoom_start=4,
    tiles='OpenStreetMap'
)

# Add circuit markers
for _, circuit in circuits.iterrows():
    folium.CircleMarker(
        location=[circuit['latitude'], circuit['longitude']],
        radius=8,
        popup=f"<b>{circuit['name']}</b><br>{circuit['city']}, {circuit['country']}",
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.7
    ).add_to(m)

# Add route lines
for idx, leg in enumerate(legs):
    folium.PolyLine(
        locations=[
            [leg.from_location.latitude, leg.from_location.longitude],
            [leg.to_location.latitude, leg.to_location.longitude]
        ],
        color='blue',
        weight=3,
        opacity=0.6,
        popup=f"<b>Leg {idx+1}</b><br>{leg.leg_name}<br>{leg.distance_km:.0f} km"
    ).add_to(m)

# Add title
title_html = '''
             <div style="position: fixed; 
                         top: 10px; left: 50px; width: 400px; height: 60px; 
                         background-color: white; border:2px solid grey; z-index:9999; 
                         font-size:16px; padding: 10px">
             <b>2019 European F1 Season Routes</b><br>
             Red markers: Circuits | Blue lines: Logistics routes
             </div>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Display map
m

## 3. Distance Analysis with Plotly

In [ ]:
# Prepare data
leg_names = [f"Leg {i+1}" for i in range(len(legs))]
distances = [leg.distance_km for leg in legs]

# Create interactive bar chart
fig = go.Figure(data=[
    go.Bar(
        x=leg_names,
        y=distances,
        marker_color='#3498db',
        text=[f"{d:.0f} km" for d in distances],
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Distance: %{y:.0f} km<extra></extra>'
    )
])

fig.update_layout(
    title='Distance by Leg - 2019 Season',
    xaxis_title='Leg',
    yaxis_title='Distance (km)',
    height=500,
    showlegend=False
)

fig.show()

## 4. Cost Breakdown Visualization

In [ ]:
# Calculate costs
params = DEFAULT_COST_PARAMS
cost_data = []

for idx, leg in enumerate(legs):
    cost = calculate_leg_cost(leg, params)
    cost_data.append({
        'Leg': f"Leg {idx+1}",
        'Fuel': cost.fuel_cost_eur,
        'Labor': cost.labor_cost_eur,
        'Tolls': cost.toll_cost_eur,
        'Fixed': cost.fixed_cost_eur
    })

df_costs = pd.DataFrame(cost_data)

# Stacked bar chart
fig = go.Figure()

fig.add_trace(go.Bar(name='Fuel', x=df_costs['Leg'], y=df_costs['Fuel'], marker_color='#e74c3c'))
fig.add_trace(go.Bar(name='Labor', x=df_costs['Leg'], y=df_costs['Labor'], marker_color='#3498db'))
fig.add_trace(go.Bar(name='Tolls', x=df_costs['Leg'], y=df_costs['Tolls'], marker_color='#2ecc71'))
fig.add_trace(go.Bar(name='Fixed', x=df_costs['Leg'], y=df_costs['Fixed'], marker_color='#f39c12'))

fig.update_layout(
    barmode='stack',
    title='Cost Breakdown by Leg',
    xaxis_title='Leg',
    yaxis_title='Cost (EUR)',
    height=500,
    hovermode='x unified'
)

fig.show()

## 5. Sunburst Chart - Cost Structure

In [ ]:
# Aggregate costs
total_fuel = df_costs['Fuel'].sum()
total_labor = df_costs['Labor'].sum()
total_tolls = df_costs['Tolls'].sum()
total_fixed = df_costs['Fixed'].sum()
grand_total = total_fuel + total_labor + total_tolls + total_fixed

# Prepare sunburst data
labels = ['Total', 'Fuel', 'Labor', 'Tolls', 'Fixed']
parents = ['', 'Total', 'Total', 'Total', 'Total']
values = [grand_total, total_fuel, total_labor, total_tolls, total_fixed]

fig = go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
    branchvalues="total",
    marker=dict(
        colors=['#ffffff', '#e74c3c', '#3498db', '#2ecc71', '#f39c12']
    ),
    hovertemplate='<b>%{label}</b><br>€%{value:,.0f}<br>%{percentParent}<extra></extra>'
))

fig.update_layout(
    title=f'Total Season Cost: €{grand_total:,.0f}',
    height=500
)

fig.show()

## 6. Emissions Heatmap

In [ ]:
# Calculate emissions
emissions_params = DEFAULT_EMISSIONS_PARAMS
emissions_data = []

for leg in legs:
    emissions = calculate_leg_emissions(leg, params['num_trucks'], emissions_params)
    emissions_data.append(emissions.total_co2e_kg / 1000)  # Convert to tonnes

# Create heatmap-style bar chart
fig = go.Figure(data=[
    go.Bar(
        x=leg_names,
        y=emissions_data,
        marker=dict(
            color=emissions_data,
            colorscale='Reds',
            showscale=True,
            colorbar=dict(title="Tonnes CO2e")
        ),
        hovertemplate='<b>%{x}</b><br>Emissions: %{y:.2f} tonnes CO2e<extra></extra>'
    )
])

fig.update_layout(
    title='CO2e Emissions by Leg',
    xaxis_title='Leg',
    yaxis_title='Emissions (tonnes CO2e)',
    height=500,
    showlegend=False
)

fig.show()

## 7. Season Summary Dashboard

In [ ]:
# Calculate summary metrics
total_distance = sum(leg.distance_km for leg in legs)
total_emissions = sum(emissions_data)

# Create subplot dashboard
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Total Distance', 'Total Cost', 'Total Emissions', 'Cost Efficiency'),
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}],
           [{'type': 'indicator'}, {'type': 'indicator'}]]
)

# Distance indicator
fig.add_trace(go.Indicator(
    mode="number",
    value=total_distance,
    number={'suffix': " km", 'valueformat': ",.0f"},
    domain={'x': [0, 1], 'y': [0, 1]}
), row=1, col=1)

# Cost indicator
fig.add_trace(go.Indicator(
    mode="number",
    value=grand_total,
    number={'prefix': "€", 'valueformat': ",.0f"},
    domain={'x': [0, 1], 'y': [0, 1]}
), row=1, col=2)

# Emissions indicator
fig.add_trace(go.Indicator(
    mode="number",
    value=total_emissions,
    number={'suffix': " t CO2e", 'valueformat': ".1f"},
    domain={'x': [0, 1], 'y': [0, 1]}
), row=2, col=1)

# Cost per km indicator
cost_per_km = grand_total / total_distance
fig.add_trace(go.Indicator(
    mode="number",
    value=cost_per_km,
    number={'prefix': "€", 'suffix': "/km", 'valueformat': ".2f"},
    domain={'x': [0, 1], 'y': [0, 1]}
), row=2, col=2)

fig.update_layout(
    title_text="2019 Season Summary Dashboard",
    height=500
)

fig.show()

## 8. Export Summary Report

In [ ]:
# Create comprehensive DataFrame
report_data = []

for idx, leg in enumerate(legs):
    cost = calculate_leg_cost(leg, params)
    emissions = calculate_leg_emissions(leg, params['num_trucks'], emissions_params)
    
    report_data.append({
        'leg_number': idx + 1,
        'from': leg.from_location.city,
        'to': leg.to_location.city,
        'distance_km': leg.distance_km,
        'fuel_cost_eur': cost.fuel_cost_eur,
        'labor_cost_eur': cost.labor_cost_eur,
        'toll_cost_eur': cost.toll_cost_eur,
        'fixed_cost_eur': cost.fixed_cost_eur,
        'total_cost_eur': cost.total_cost_eur,
        'co2e_tonnes': emissions.total_co2e_kg / 1000
    })

df_report = pd.DataFrame(report_data)

# Display sample
print("Sample Report Data:\n")
print(df_report.head(10).to_string(index=False))

# Save to CSV
output_file = '../data/reports/season_2019_analysis.csv'
# df_report.to_csv(output_file, index=False)
# print(f"\nFull report saved to: {output_file}")

## Summary

This notebook demonstrated:
- Interactive geographic maps with Folium
- Rich interactive charts with Plotly
- Multi-dimensional data visualization (distance, cost, emissions)
- Dashboard-style summary views
- Export capabilities for reports

**Visualization Techniques**:
- Geographic: Circuit markers, route polylines
- Comparison: Bar charts, stacked bars
- Hierarchical: Sunburst charts
- Continuous: Color-scaled heatmaps
- Summary: Indicator gauges

**Next Steps**:
- Integrate into Streamlit dashboard
- Add user interactivity (parameter adjustment)
- Real-time updates
- Multi-season comparisons